# Season 20 Election Notes

>**FORBIDDEN KNOWLEDGE WARNING**
>
>This document contains some information that is not currently present on the main site and may be considered spoilers.
>Continue at your own discretion. Some information in this document cannot be shared on the Discord without spoiler tags.

In [1]:
import pandas
%matplotlib inline
from blaseball_mike.models import *
from blaseball_mike.tables import StatType
import plotly.express as plot
import plotly.io as _pio
import plotly.subplots as subplot
from IPython.display import display, Markdown
from copy import deepcopy

import os
import sys
pdir = os.path.abspath(os.path.join(os.path.dirname(''), os.path.pardir))
sys.path.append(pdir)
from display import *
from blessings import *
sys.path.remove(pdir)

_pio.renderers.default = "notebook_connected"

pies = Team.load_by_name("Philly Pies")

# Fix Attractors
real_pies = deepcopy(pies)
for p in real_pies.lineup + real_pies.rotation:
    p.batting_rating = None
    p.pitching_rating = None
    p.baserunning_rating = None
    p.defense_rating = None

In [2]:
sim = SimulationData.load()
display(Markdown(f"**Last Updated Season {sim.season}, Day {sim.day}**"))

**Last Updated Season 20, Day 27**

---
## Decrees

Decrees are chosen by majority votes across all teams. `1` decrees will pass this season.

### Sun 3
**Set Sun. If your Team sweeps all 3 games of a Regular Season series, the final non-loss is worth 2 Wins.**

See below for the number of additional Wins this would have added. It is unknown how this interacts with the current
Turntables/SunSun mess.

In [3]:
extra_wins = 0
s19_games = Game.load_by_season(season=19, team_id=pies.id)
for i in range(0, len(s19_games), 3):
    series = list(s19_games.values())[i:i+3]
    if series[0].winning_team.id == pies.id and series[1].winning_team.id == pies.id and series[1].winning_team.id == pies.id:
        extra_wins += 1
print(f"Pies would have an extra {extra_wins} Wins")

Pies would have an extra 16 Wins


### Sun 9

**Set Sun. Non-losses on the Week End (every 9th Day) are worth 2 Wins.**

See below for the number of additional Wins this would have added. It is unknown how this interacts with the current
Turntables/SunSun mess.

In [4]:
extra_wins = 0
for g in s19_games.values():
    if (g.day - 1) % 9 == 8 and g.day <= 99:
        if g.winning_team.id == pies.id:
            extra_wins += 1
print(f"Pies would have an extra {extra_wins} Wins")

Pies would have an extra 11 Wins


### Sun 30

**Set Sun. When a game enters Extra Innings, both Teams will immediately earn 1 Win.**

See below for the number of additional Wins this would have added. It is unknown how this interacts with the current
Turntables/SunSun mess.

In [5]:
extra_wins = 0
for g in s19_games.values():
    if g.inning > 9:
        extra_wins += 1
print(f"Pies would have an extra {extra_wins} Wins")

Pies would have an extra 4 Wins


___
## Wills

Will are selected for each team by raffle. Each team will draw `2` random wills. The bottom 4 teams in the league by
standings will draw `3`. The chosen wills are random, but wills with more votes are more likely to be selected.

### Equivalent Exchange
**Swap a Player from a Team in the League with an Equivalent (within 2 Stars) Player on your Team.**

See below for the 20 best swap candidates for a particular player, sorted by highest stats in the applicable position.

In [6]:
league = League.load()
all_teams = list(league.teams.values())
all_active = []
for team in all_teams:
    if team.id != pies.id:
        all_active.extend(team.lineup + team.rotation)
all_total_stars = get_total_stars(all_active)

In [7]:
for player in pies.lineup:
    total_stars = get_total_stars(player)[player.id]
    valid_total_stars = {k: v for k, v in all_total_stars.items() if total_stars-2 < v < total_stars+2}
    player_matches = [x for x in all_active if x.id in valid_total_stars.keys()]
    player_matches.sort(key=lambda x: x.batting_rating, reverse=True)
    display(Markdown(f"####{player.name}"))
    display(get_stars([player] + player_matches[0:20], include_team=True))
    display(set_heatmap(get_batting_stlats([player] + player_matches[0:20]), maxVal=1.5))

####Eduardo Woodman

,,Combined,Batting,Pitching,Baserunning,Defense
,Team,,,,,
Eduardo Woodman,Pies,14.3,5.6,1.8,3.7,3.3
Howell Franklin,Sunbeams,13.0,6.1,1.5,2.6,2.7
Jacob Haynes,Flowers,13.0,5.6,2.9,2.5,1.9
Alston Cerveza,Crabs,14.2,5.5,3.0,2.6,3.1
Dudl-y Mu-ller,Spies,16.2,5.4,3.1,3.3,4.5
Margarito Nava,Flowers,16.2,5.4,3.7,4.4,2.7
Commissioner Vapor,Moist Talkers,15.8,5.3,3.1,4.4,3.0
Fish Summer,Tacos,12.8,5.2,1.8,3.6,2.3
Scores Baserunner,Flowers,16.0,5.1,2.5,3.8,4.7


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,1.126442,1.345315,1.152985,0.970888,0.876160,0.283687,0.749895,0.001000,1.025183
Howell Franklin,1.226470,1.107821,1.353333,1.712056,1.274572,0.001000,1.217685,0.001000,1.402964
Jacob Haynes,1.126749,1.334891,1.020797,1.141949,1.265377,0.001000,0.440711,0.013502,0.883104
Alston Cerveza,1.103390,1.265764,0.964120,1.163440,1.251674,0.065767,1.319157,0.151105,0.611425
Dudl-y Mu-ller,1.079778,1.644472,0.776783,0.987971,1.021591,0.182529,1.023215,0.001000,1.719870
Margarito Nava,1.072049,0.867560,1.315733,1.320216,1.456526,0.444288,1.197500,0.001000,0.858627
Commissioner Vapor,1.056348,1.182365,1.008728,0.687493,1.265242,0.001000,1.201027,0.001000,0.893326
Fish Summer,1.036398,1.053025,1.122064,0.876586,0.903681,0.001000,0.774672,0.001000,1.224700
Scores Baserunner,1.020751,1.255932,1.044977,0.719606,0.603396,0.001000,0.549101,0.001000,0.482450
G-a Holb--ok,1.013860,1.396818,0.752736,1.133225,0.902763,0.054784,0.875004,0.001000,1.251697


####Beasley Day

,,Combined,Batting,Pitching,Baserunning,Defense
,Team,,,,,
Beasley Day,Pies,14.2,4.5,3.0,3.6,3.0
Howell Franklin,Sunbeams,13.0,6.1,1.5,2.6,2.7
Jacob Haynes,Flowers,13.0,5.6,2.9,2.5,1.9
Alston Cerveza,Crabs,14.2,5.5,3.0,2.6,3.1
Dudl-y Mu-ller,Spies,16.2,5.4,3.1,3.3,4.5
Margarito Nava,Flowers,16.2,5.4,3.7,4.4,2.7
Commissioner Vapor,Moist Talkers,15.8,5.3,3.1,4.4,3.0
Fish Summer,Tacos,12.8,5.2,1.8,3.6,2.3
Scores Baserunner,Flowers,16.0,5.1,2.5,3.8,4.7


...

,,Combined,Batting,Pitching,Baserunning,Defense
,Team,,,,,
Peanut Holloway,Pies,10.8,4.2,1.1,2.2,3.4
Agan Harrison,Shoe Thieves,12.0,5.2,1.6,3.3,1.8
Fish Summer,Tacos,12.8,5.2,1.8,3.6,2.3
Summers Preston,Dale,11.0,5.1,1.9,1.9,2.1
Rat Mason,Tacos,10.2,5.0,1.3,2.4,1.5
Lotus Mango,Garages,12.8,4.4,2.8,2.6,2.9
Kit Honey,Lift,10.3,4.4,1.3,2.4,2.2
Helga Washington,Breath Mints,12.1,4.4,2.4,1.9,3.5
Randy Castillo,Tigers,10.1,4.3,1.7,1.9,2.2


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Peanut Holloway,0.848814,0.769919,0.943909,0.997017,0.943055,0.006915,0.197960,0.773163,0.974957
Agan Harrison,1.048231,1.229399,1.202765,0.844173,0.762802,0.596828,0.630691,0.193311,1.161214
Fish Summer,1.036398,1.053025,1.122064,0.876586,0.903681,0.001000,0.774672,0.001000,1.224700
Summers Preston,1.015975,0.945562,1.285928,1.267384,0.403316,0.001000,0.896441,0.001000,0.816163
Rat Mason,1.009065,1.290628,0.798484,1.077502,0.906716,0.030115,1.092972,0.001000,0.653243
Lotus Mango,0.884298,0.871670,1.104360,0.518383,0.709745,0.132498,0.252524,0.001000,0.562986
Kit Honey,0.880170,1.018119,0.689806,1.238424,0.986700,0.227591,0.738367,0.001000,0.416743
Helga Washington,0.870301,1.032383,0.937105,0.575477,0.447197,0.106190,0.373174,0.020668,0.643879
Randy Castillo,0.868039,1.030142,1.096949,0.686627,0.257003,0.629329,0.804416,0.022455,1.085623
Tiana Wheeler,0.855632,0.916610,0.604948,1.067869,0.140479,0.206921,0.835125,0.261758,0.422096


####Usurper Violet

,,Combined,Batting,Pitching,Baserunning,Defense
,Team,,,,,
Usurper Violet,Pies,11.7,2.9,3.2,1.7,3.9
Howell Franklin,Sunbeams,13.0,6.1,1.5,2.6,2.7
Jacob Haynes,Flowers,13.0,5.6,2.9,2.5,1.9
Agan Harrison,Shoe Thieves,12.0,5.2,1.6,3.3,1.8
Fish Summer,Tacos,12.8,5.2,1.8,3.6,2.3
Summers Preston,Dale,11.0,5.1,1.9,1.9,2.1
Rat Mason,Tacos,10.2,5.0,1.3,2.4,1.5
Eugenia Garbage,Moist Talkers,13.1,4.6,4.3,2.0,2.1
Denzel Scott,Spies,12.9,4.5,3.1,2.6,2.7


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Usurper Violet,0.579715,0.894648,0.293503,1.022726,0.869795,0.569699,0.283561,0.100000,0.487176
Howell Franklin,1.226470,1.107821,1.353333,1.712056,1.274572,0.001000,1.217685,0.001000,1.402964
Jacob Haynes,1.126749,1.334891,1.020797,1.141949,1.265377,0.001000,0.440711,0.013502,0.883104
Agan Harrison,1.048231,1.229399,1.202765,0.844173,0.762802,0.596828,0.630691,0.193311,1.161214
Fish Summer,1.036398,1.053025,1.122064,0.876586,0.903681,0.001000,0.774672,0.001000,1.224700
Summers Preston,1.015975,0.945562,1.285928,1.267384,0.403316,0.001000,0.896441,0.001000,0.816163
Rat Mason,1.009065,1.290628,0.798484,1.077502,0.906716,0.030115,1.092972,0.001000,0.653243
Eugenia Garbage,0.917001,1.273527,0.875403,0.409978,0.440177,0.001000,1.207960,0.001000,1.143672
Denzel Scott,0.906225,1.005499,0.925789,0.609571,0.704572,0.076566,0.740125,0.001000,0.998163
Oliver Loofah,0.886196,1.014675,0.948006,0.386613,0.864513,0.065929,0.348519,0.052834,0.292862


In [8]:
for player in pies.rotation:
    total_stars = get_total_stars(player)[player.id]
    valid_total_stars = {k: v for k, v in all_total_stars.items() if total_stars-2 < v < total_stars+2}
    player_matches = [x for x in all_active if x.id in valid_total_stars.keys()]
    player_matches.sort(key=lambda x: x.pitching_rating, reverse=True)
    display(Markdown(f"####{player.name}"))
    display(get_stars([player] + player_matches[0:20], include_team=True))
    display(set_heatmap(get_pitching_stlats([player] + player_matches[0:20]), maxVal=1.5))

####Henry Marshallow

,,Combined,Batting,Pitching,Baserunning,Defense
,Team,,,,,
Henry Marshallow,Pies,14.8,3.6,3.8,2.7,4.7
Montgomery Bullock,Crabs,14.7,1.6,6.6,2.4,4.0
Sigmund Castillo,Sunbeams,16.2,3.3,6.4,4.8,1.6
Juice Collins,Fridays,16.5,2.8,6.2,4.1,3.4
Cell Barajas,Wild Wings,15.8,3.6,5.9,2.9,3.4
Patty Fox,Millennials,15.1,4.6,5.7,1.6,3.2
Inky Rutledge,Lift,13.8,2.2,5.5,3.2,2.9
Zoey Kirchner,Steaks,15.0,3.2,5.2,4.0,2.6
Sandford Garner,Lovers,15.7,4.1,5.1,3.4,3.1


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.768310,0.491270,1.171077,1.428524,0.822605,0.808351,0.876106
Montgomery Bullock,1.318505,1.738706,0.999277,1.152921,0.742616,1.412281,0.908350
Sigmund Castillo,1.289484,1.302577,1.068161,1.400424,1.429219,1.460265,1.094387
Juice Collins,1.245491,1.279794,1.257108,0.943706,1.266897,0.662010,1.331051
Cell Barajas,1.182545,1.206373,1.255300,1.046099,0.970262,0.434633,0.410170
Patty Fox,1.141465,1.306372,1.210541,0.678718,0.935361,0.596270,0.747740
Inky Rutledge,1.092614,1.515457,0.864538,0.624700,1.113789,0.950643,1.082086
Zoey Kirchner,1.046886,1.017730,1.117259,0.895819,1.088866,1.027149,0.649715
Sandford Garner,1.020016,1.078199,0.787564,1.542840,1.173474,0.874803,1.380823
Lorcan Smaht,1.012278,1.221527,0.822904,1.084371,0.916908,0.586109,1.181545


####Jacoby Podcast

,,Combined,Batting,Pitching,Baserunning,Defense
,Team,,,,,
Jacoby Podcast,Pies,12.7,2.7,4.8,1.8,3.5
Montgomery Bullock,Crabs,14.7,1.6,6.6,2.4,4.0
Inky Rutledge,Lift,13.8,2.2,5.5,3.2,2.9
Bennett Bluesky,Spies,11.7,2.1,5.4,1.6,2.7
Silvia Rugrat,Wild Wings,12.7,3.2,5.2,1.8,2.5
Elvis Figueroa,Sunbeams,14.1,3.1,5.0,1.8,4.3
Mummy Melcon,Tigers,14.7,3.9,5.0,2.8,3.1
Erin Jesaulenko,Georgias,13.0,1.5,4.9,2.3,4.2
Sixpack Santiago,Dale,14.2,2.1,4.8,4.4,2.9


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Jacoby Podcast,0.954583,0.824371,1.141648,0.868777,1.157278,1.154442,0.513636
Montgomery Bullock,1.318505,1.738706,0.999277,1.152921,0.742616,1.412281,0.908350
Inky Rutledge,1.092614,1.515457,0.864538,0.624700,1.113789,0.950643,1.082086
Bennett Bluesky,1.072733,1.300206,1.057101,0.585718,0.937581,1.146555,1.079501
Silvia Rugrat,1.039169,1.071044,1.229974,0.662651,0.800881,1.233076,1.361413
Elvis Figueroa,0.997790,1.061528,1.069539,1.011878,0.700624,0.365663,0.615701
Mummy Melcon,0.991416,0.719798,1.292408,1.088140,1.441466,1.170313,1.050421
Erin Jesaulenko,0.980962,1.208308,1.093363,0.409752,0.908798,0.782202,0.693528
Sixpack Santiago,0.969910,0.765921,1.199201,1.531857,0.747720,0.826116,0.559834
Mindy Salad,0.968347,0.871755,1.229726,0.977278,0.705356,0.728920,0.345044


####Lucy Tokkan

,,Combined,Batting,Pitching,Baserunning,Defense
,Team,,,,,
Lucy Tokkan,Pies,9.9,2.1,3.7,1.1,2.9
Dunlap Figueroa,Tigers,8.7,1.1,5.5,1.0,1.1
Bennett Bluesky,Spies,11.7,2.1,5.4,1.6,2.7
Cudi Di Batterino,Moist Talkers,11.3,1.9,4.8,1.3,3.3
Wyatt Mason IV,Steaks,11.5,1.1,4.5,2.1,3.8
Orville Manco,Steaks,11.2,2.9,4.5,1.8,2.0
Cravel Gesundheit,Magic,9.9,1.6,4.2,1.6,2.4
Beasley Gloom,Moist Talkers,9.8,1.2,4.2,2.6,1.9
Hiroto Wilcox,Tigers,8.2,1.0,4.1,1.0,2.1


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Lucy Tokkan,0.741748,0.901966,0.995562,0.263758,0.973020,0.180980,0.321805
Dunlap Figueroa,1.096410,1.126199,1.303006,0.487574,1.458415,0.878442,1.172395
Bennett Bluesky,1.072733,1.300206,1.057101,0.585718,0.937581,1.146555,1.079501
Cudi Di Batterino,0.956106,0.964983,0.954493,1.308338,0.581326,1.245921,0.583496
Wyatt Mason IV,0.909013,0.859159,1.110696,0.885281,0.977619,0.194373,0.879172
Orville Manco,0.906168,1.124421,0.904682,0.931748,0.376287,0.704634,0.998314
Cravel Gesundheit,0.847828,0.869140,0.987573,0.784254,0.634989,0.723778,0.086046
Beasley Gloom,0.834387,1.132365,0.956163,0.528743,0.329751,0.472345,0.437574
Hiroto Wilcox,0.811632,0.948969,1.048703,0.344252,0.712457,0.735528,0.281270
Cornelius Games,0.798203,0.998508,0.969635,0.926127,0.177550,0.326936,0.567173


### Swap
**Swap two Players on your Team's Roster.**

This allows switching the position of any two players, including from the shadows. Because the most complex iteration of
this is shadows swaps, see below for that analysis.

**Batting:**

In [9]:
shadows = real_pies.shadows
shadows.sort(key=lambda x: x.batting_rating, reverse=True)
best_batting_shadows = shadows[0:4]
worst_batter = sort_lineup(pies, 1)[0]

display(get_stars([worst_batter] + best_batting_shadows))
display(set_heatmap(get_batting_stlats([worst_batter] + best_batting_shadows), maxVal=1))

,Combined,Batting,Pitching,Baserunning,Defense
Usurper Violet,11.7,2.9,3.2,1.7,3.9
Doc Anice,15.6,4.4,3.7,3.2,4.3
Haruta Byrd,10.5,3.6,2.7,2.4,1.7
Javier Lotus,11.4,3.4,0.8,3.3,3.9
August Obrien,9.8,3.4,1.0,3.3,2.0


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Usurper Violet,0.579715,0.894648,0.293503,1.022726,0.869795,0.569699,0.283561,0.100000,0.487176
Doc Anice,0.874037,1.009565,0.675395,0.959085,0.991442,0.001000,1.230245,0.087928,1.012157
Haruta Byrd,0.724391,0.721433,0.917077,0.853010,0.585921,0.832203,0.169263,0.100000,0.124161
Javier Lotus,0.682621,0.938964,0.386700,0.901328,0.942211,0.205027,0.884526,0.100000,0.815592
August Obrien,0.681669,0.641217,0.927569,0.913891,0.492708,0.923854,0.556849,0.100000,0.463958


**Pitching:**

In [10]:
shadows.sort(key=lambda x: x.pitching_rating, reverse=True)
best_pitching_shadows = shadows[0:4]
worst_pitcher = sort_rotation(pies, 1)[0]

display(get_stars([worst_pitcher] + best_pitching_shadows))
display(set_heatmap(get_pitching_stlats([worst_pitcher] + best_pitching_shadows), maxVal=1))

,Combined,Batting,Pitching,Baserunning,Defense
Lucy Tokkan,9.9,2.1,3.7,1.1,2.9
Tiana Takahashi,12.6,1.7,4.1,3.1,3.8
Doc Anice,15.6,4.4,3.7,3.2,4.3
Twofurious Puddles,11.5,2.6,3.5,1.8,3.6
Case Lancaster,8.9,0.2,3.3,2.2,3.2


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Lucy Tokkan,0.741748,0.901966,0.995562,0.263758,0.973020,0.180980,0.321805
Tiana Takahashi,0.812669,0.959448,1.018362,1.015177,0.153260,0.705865,1.024250
Doc Anice,0.734279,1.076739,0.487964,0.959145,0.632001,0.765347,0.305586
Twofurious Puddles,0.694922,0.811110,0.650034,0.807712,0.718615,0.416240,0.725894
Case Lancaster,0.658365,0.867044,0.829197,0.190119,0.825333,0.868251,0.082117


### Move
**Move a Player on your Team to the desired location on your Roster.**

Move would move a player from any location on the team (including Shadows) to another position, increasing the number
of players in that position. See below for moving various players into different positions.

**Batting**

In [11]:
bhp = best_hitting_pitcher(real_pies)
bph = best_pitching_hitter(real_pies)
worst_batter = sort_lineup(real_pies, num=1)[0]
worst_pitcher = sort_rotation(real_pies, num=1)[0]

display(Markdown(f"Moving {bhp.name} to the Lineup"))
display(pandas.DataFrame(move_player(pies, bhp, "lineup")))

display(Markdown(f"Moving {worst_batter.name} to the Shadows"))
display(pandas.DataFrame(move_player(pies, worst_batter, "shadows")))

display(Markdown(f"Moving {best_batting_shadows[0].name} to the Lineup"))
display(pandas.DataFrame(move_player(pies, best_batting_shadows[0], "lineup")))

Moving Henry Marshallow to the Lineup

,Pies
batting_change,-2.1422%
pitching_change,+2.6618%
baserunning_change,-1.5418%
defense_change,+2.3973%


Moving Usurper Violet to the Shadows

,Pies
batting_change,+4.6626%
pitching_change,+0.0000%
baserunning_change,+5.0205%
defense_change,-0.8942%


Moving Doc Anice to the Lineup

,Pies
batting_change,-0.3562%
pitching_change,+0.0000%
baserunning_change,-0.4499%
defense_change,+1.6097%


**Pitching**

In [12]:
display(Markdown(f"Moving {bph.name} to the Rotation"))
display(pandas.DataFrame(move_player(pies, bph, "rotation")))

display(Markdown(f"Moving {worst_pitcher.name} to the Shadows"))
display(pandas.DataFrame(move_player(pies, worst_pitcher, "shadows")))

display(Markdown(f"Moving {best_pitching_shadows[0].name} to the Rotation"))
display(pandas.DataFrame(move_player(pies, best_pitching_shadows[0], "rotation")))

Moving Ruslan Greatness to the Rotation

,Pies
batting_change,-0.6429%
pitching_change,-0.0552%
baserunning_change,-4.7962%
defense_change,-4.2053%


Moving Lucy Tokkan to the Shadows

,Pies
batting_change,+0.0000%
pitching_change,+3.9899%
baserunning_change,+0.0000%
defense_change,+0.0000%


Moving Tiana Takahashi to the Rotation

,Pies
batting_change,+0.0000%
pitching_change,-0.2219%
baserunning_change,+0.0000%
defense_change,+0.0000%


### Item Steal
**Take an Item, repair it, and move it to a Player on your Team.**

This would steal an item from another team and give it to a chosen player on our team. The player receiving the item
will drop a random item if there are no open item slots. See below for a list of all non-base items held by players in
the league.

In [13]:
players = Player.load_all()
carried_items = []
for p in players.values():
    if getattr(p,"_league_team_id", None) == pies.id:
        continue
    for item in p.items:
        if len(item.adjustments) > 1:
            carried_items.append((p, item))

table = []
for p, i in carried_items:
    mods = [Modification.load_one(a["mod"]).title for a in i.adjustments if a["type"] == 0]
    table.append(pandas.Series({"Wielder": p.name, "Team": p.league_team.nickname, "Max Durability": i.durability, "Modifications": mods}, name=i.name))
pandas.DataFrame(table)

,Wielder,Team,Max Durability,Modifications
Chunky Bat,Jomgy Rolsenthal,Dale,2,[Chunky]
Chaotic Shoes,Simon Haley,Shoe Thieves,2,[]
Socks of Intelligence,Glabe Moon,Flowers,2,[]
Cool Socks,Jaylen Hotdogfingers,Flowers,2,[]
Hard Ring,Xandra Pancakes,Worms,2,[]
Parasitic Shoes,Scratch Deleuze,Worms,2,[Parasite]
Gravity Greedy Fire Eating Socks,Scratch Deleuze,Worms,4,"[Gravity, Fire Eater]"
Wooden Jersey,Curry Aliciakeyes,Magic,2,[]
Fireproof Jersey of Intelligence,Sexton Wheerer,Tacos,3,[Fireproof]
Cool Cap,Michelle Sportsman,Tacos,2,[]


### Shadow Infuse
**Boost a random Stat Category of a Player in your Shadows by 20% to 40%.**

See below for how this would affect various team members at the 2 extreme percentages. This will only affect one player.

#### Batting

In [14]:
display(Markdown("**20% Change**"))
infuse_batting_table, _, _ = improve_team_batting_table(real_pies, 0.20, shadows=True)
display(infuse_batting_table)
display(set_heatmap(get_batting_stlats(improve_team_batting(real_pies, 0.20, shadows=True)), maxVal=1))

**20% Change**

,old_batting_stars,new_batting_stars,change_in_batting_stars
Tiana Takahashi,1.670798,2.871259,1.200461
Doc Anice,4.370186,5.260440,0.890254
Twofurious Puddles,2.635533,3.759716,1.124183
Case Lancaster,0.183966,2.432678,2.248712
Castillo Logan,1.858396,3.240490,1.382094
Haruta Byrd,3.621954,4.642808,1.020854
Yusef Puddles,2.390547,3.545751,1.155204
Mummy Cantburn,1.390958,2.681958,1.291000
Sheev Shriffle,3.352957,4.372160,1.019204
Doginic Burgertoes,2.079285,3.353370,1.274085


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Tiana Takahashi,0.574252,0.642405,0.310022,1.285608,1.137718,0.205743,0.711365,0.010000,0.420455
Doc Anice,1.052088,1.209565,0.875395,1.159085,1.191442,0.010000,1.430245,0.010000,1.212157
Twofurious Puddles,0.751943,0.777625,0.680623,1.057061,1.176273,0.612102,0.209182,0.010000,0.674513
Case Lancaster,0.486536,0.941388,0.200246,0.273538,0.850109,0.010000,0.266786,0.010000,0.450802
Castillo Logan,0.648098,0.498816,0.828983,0.933602,0.207738,0.010000,0.950237,0.010000,0.421515
Haruta Byrd,0.928562,0.921433,1.117077,1.053010,0.785921,0.632203,0.369263,0.010000,0.324161
Yusef Puddles,0.709150,1.268932,0.423433,0.902603,0.391137,0.419418,0.351715,0.010000,1.250129
Mummy Cantburn,0.536392,0.295592,0.776609,1.147265,0.725890,0.716564,0.212553,0.010000,1.127160
Sheev Shriffle,0.874432,1.184880,0.810356,0.421637,0.682530,0.402199,0.974400,0.029204,0.753719
Doginic Burgertoes,0.670674,0.376724,1.184426,0.305629,1.134521,0.012919,0.238098,0.561237,0.504905


In [15]:
display(Markdown("**40% Change**"))
infuse_batting_table, _, _ = improve_team_batting_table(real_pies, 0.40, shadows=True)
display(infuse_batting_table)
display(set_heatmap(get_batting_stlats(improve_team_batting(real_pies, 0.40, shadows=True)), maxVal=1))

**40% Change**

,old_batting_stars,new_batting_stars,change_in_batting_stars
Tiana Takahashi,1.670798,3.907154,2.236356
Doc Anice,4.370186,6.122589,1.752403
Twofurious Puddles,2.635533,4.728474,2.092941
Case Lancaster,0.183966,3.550183,3.366217
Castillo Logan,1.858396,4.213635,2.355239
Haruta Byrd,3.621954,5.596436,1.974482
Yusef Puddles,2.390547,4.582121,2.191574
Mummy Cantburn,1.390958,3.735507,2.344550
Sheev Shriffle,3.352957,5.332400,1.979444
Doginic Burgertoes,2.079285,4.391437,2.312151


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Tiana Takahashi,0.781431,0.842405,0.510022,1.485608,1.337718,0.010000,0.911365,0.010000,0.620455
Doc Anice,1.224518,1.409565,1.075395,1.359085,1.391442,0.010000,1.630245,0.010000,1.412157
Twofurious Puddles,0.945695,0.977625,0.880623,1.257061,1.376273,0.412102,0.409182,0.010000,0.874513
Case Lancaster,0.710037,1.141388,0.400246,0.473538,1.050109,0.010000,0.466786,0.010000,0.650802
Castillo Logan,0.842727,0.698816,1.028983,1.133602,0.407738,0.010000,1.150237,0.010000,0.621515
Haruta Byrd,1.119287,1.121433,1.317077,1.253010,0.985921,0.432203,0.569263,0.010000,0.524161
Yusef Puddles,0.916424,1.468932,0.623433,1.102603,0.591137,0.219418,0.551715,0.010000,1.450129
Mummy Cantburn,0.747101,0.495592,0.976609,1.347265,0.925890,0.516564,0.412553,0.010000,1.327160
Sheev Shriffle,1.066480,1.384880,1.010356,0.621637,0.882530,0.202199,1.174400,0.010000,0.953719
Doginic Burgertoes,0.878287,0.576724,1.384426,0.505629,1.334521,0.010000,0.438098,0.361237,0.704905


#### Pitching

In [16]:
display(Markdown("**20% Change**"))
infuse_pitching_table, _, _ = improve_team_pitching_table(real_pies, 0.20, shadows=True)
display(infuse_pitching_table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(real_pies, 0.20, shadows=True)), maxVal=1))

**20% Change**

,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Tiana Takahashi,4.063346,5.393097,1.329751
Doc Anice,3.671394,4.878623,1.207230
Twofurious Puddles,3.474609,4.620631,1.146021
Case Lancaster,3.291826,4.555580,1.263754
Castillo Logan,3.022344,4.346669,1.324325
Haruta Byrd,2.728771,3.927099,1.198328
Yusef Puddles,2.508852,3.633514,1.124663
Mummy Cantburn,2.326166,3.450122,1.123956
Sheev Shriffle,1.942316,3.092375,1.150060
Doginic Burgertoes,1.808231,2.965528,1.157297


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Tiana Takahashi,1.078619,1.159448,1.218362,1.215177,0.353260,0.905865,1.224250
Doc Anice,0.975725,1.276739,0.687964,1.159145,0.832001,0.965347,0.505586
Twofurious Puddles,0.924126,1.011110,0.850034,1.007712,0.918615,0.616240,0.925894
Case Lancaster,0.911116,1.067044,1.029197,0.390119,1.025333,1.068251,0.282117
Castillo Logan,0.869334,1.162056,0.879044,0.366984,1.194242,0.290067,0.430832
Haruta Byrd,0.785420,0.946009,0.884712,0.389109,0.802043,0.955252,0.560515
Yusef Puddles,0.726703,0.769080,0.677310,1.233877,0.525880,1.025805,0.817095
Mummy Cantburn,0.690024,0.823505,0.792185,0.439305,0.637026,0.612072,1.172225
Sheev Shriffle,0.618475,0.463763,1.009104,0.679938,0.644447,1.077531,0.841488
Doginic Burgertoes,0.593106,0.789539,0.414470,1.113112,0.511613,0.961430,0.220448


In [17]:
display(Markdown("**40% Change**"))
infuse_pitching_table, _, _ = improve_team_pitching_table(real_pies, 0.40, shadows=True)
display(infuse_pitching_table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(real_pies, 0.40, shadows=True)), maxVal=1))

**40% Change**

,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Tiana Takahashi,4.063346,6.673802,2.610456
Doc Anice,3.671394,6.109408,2.438014
Twofurious Puddles,3.474609,5.807659,2.333050
Case Lancaster,3.291826,5.797771,2.505946
Castillo Logan,3.022344,5.610596,2.588252
Haruta Byrd,2.728771,5.127135,2.398363
Yusef Puddles,2.508852,4.799377,2.290526
Mummy Cantburn,2.326166,4.608101,2.281935
Sheev Shriffle,1.942316,4.265267,2.322952
Doginic Burgertoes,1.808231,4.136959,2.328728


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Tiana Takahashi,1.334760,1.359448,1.418362,1.415177,0.553260,1.105865,1.424250
Doc Anice,1.221882,1.476739,0.887964,1.359145,1.032001,1.165347,0.705586
Twofurious Puddles,1.161532,1.211110,1.050034,1.207712,1.118615,0.816240,1.125894
Case Lancaster,1.159554,1.267044,1.229197,0.590119,1.225333,1.268251,0.482117
Castillo Logan,1.122119,1.362056,1.079044,0.566984,1.394242,0.490067,0.630832
Haruta Byrd,1.025427,1.146009,1.084712,0.589109,1.002043,1.155252,0.760515
Yusef Puddles,0.959875,0.969080,0.877310,1.433877,0.725880,1.225805,1.017095
Mummy Cantburn,0.921620,1.023505,0.992185,0.639305,0.837026,0.812072,1.372225
Sheev Shriffle,0.853053,0.663763,1.209104,0.879938,0.844447,1.277531,1.041488
Doginic Burgertoes,0.827392,0.989539,0.614470,1.313112,0.711613,1.161430,0.420448


#### Baserunning

In [18]:
display(Markdown("**20% Change**"))
infuse_baserunning_table, _, _ = improve_team_baserunning_table(real_pies, 0.20, shadows=True)
display(infuse_baserunning_table)
display(set_heatmap(get_baserunning_stlats(improve_team_baserunning(real_pies, 0.20, shadows=True)), maxVal=1))

**20% Change**

,old_baserunning_stars,new_baserunning_stars,change_in_baserunning_stars
Tiana Takahashi,3.110963,4.104404,0.993441
Doc Anice,3.229280,4.433390,1.204110
Twofurious Puddles,1.817993,2.913054,1.095061
Case Lancaster,2.215962,3.275592,1.059631
Castillo Logan,2.497486,3.464649,0.967163
Haruta Byrd,2.445376,3.562717,1.117341
Yusef Puddles,2.938345,3.927098,0.988753
Mummy Cantburn,2.021513,3.133820,1.112307
Sheev Shriffle,2.215166,3.254024,1.038858
Doginic Burgertoes,1.779564,3.023082,1.243518


,Baserunning Rating,Laserlikeness,Continuation,Base Thirst,Indulgence,Ground Friction
Tiana Takahashi,0.820881,0.904916,0.541599,0.904336,0.442265,1.056988
Doc Anice,0.886678,1.251547,0.365112,0.884669,0.966406,0.313369
Twofurious Puddles,0.582611,0.393390,0.578033,0.714860,1.088752,1.063079
Case Lancaster,0.655118,0.592932,0.689273,0.875411,0.309200,1.064958
Castillo Logan,0.692930,0.652448,0.582011,0.700285,0.965487,0.548541
Haruta Byrd,0.712543,0.695418,0.964453,0.905069,0.272388,0.872433
Yusef Puddles,0.785420,0.831219,0.449357,1.233696,0.696257,0.583273
Mummy Cantburn,0.626764,0.488250,1.032320,1.139393,0.298762,0.959435
Sheev Shriffle,0.650805,0.473349,0.818521,1.011066,1.171078,0.591838
Doginic Burgertoes,0.604616,0.522288,0.227095,0.817854,0.827448,1.093013


In [19]:
display(Markdown("**40% Change**"))
infuse_baserunning_table, _, _ = improve_team_baserunning_table(real_pies, 0.40, shadows=True)
display(infuse_baserunning_table)
display(set_heatmap(get_baserunning_stlats(improve_team_baserunning(real_pies, 0.40, shadows=True)), maxVal=1))

**40% Change**

,old_baserunning_stars,new_baserunning_stars,change_in_baserunning_stars
Tiana Takahashi,3.110963,5.042735,1.931772
Doc Anice,3.229280,5.449501,2.220221
Twofurious Puddles,1.817993,3.908728,2.090735
Case Lancaster,2.215962,4.241752,2.025790
Castillo Logan,2.497486,4.396789,1.899303
Haruta Byrd,2.445376,4.533160,2.087784
Yusef Puddles,2.938345,4.866021,1.927676
Mummy Cantburn,2.021513,4.128766,2.107253
Sheev Shriffle,2.215166,4.224486,2.009320
Doginic Burgertoes,1.779564,4.022373,2.242810


,Baserunning Rating,Laserlikeness,Continuation,Base Thirst,Indulgence,Ground Friction
Tiana Takahashi,1.008547,1.104916,0.741599,1.104336,0.642265,1.256988
Doc Anice,1.089900,1.451547,0.565112,1.084669,1.166406,0.513369
Twofurious Puddles,0.781746,0.593390,0.778033,0.914860,1.288752,1.263079
Case Lancaster,0.848350,0.792932,0.889273,1.075411,0.509200,1.264958
Castillo Logan,0.879358,0.852448,0.782011,0.900285,1.165487,0.748541
Haruta Byrd,0.906632,0.895418,1.164453,1.105069,0.472388,1.072433
Yusef Puddles,0.973204,1.031219,0.649357,1.433696,0.896257,0.783273
Mummy Cantburn,0.825753,0.688250,1.232320,1.339393,0.498762,1.159435
Sheev Shriffle,0.844897,0.673349,1.018521,1.211066,1.371078,0.791838
Doginic Burgertoes,0.804475,0.722288,0.427095,1.017854,1.027448,1.293013


#### Defense

In [20]:
display(Markdown("**20% Change**"))
infuse_defense_table, _, _ = improve_team_defense_table(real_pies, 0.20, shadows=True)
display(infuse_defense_table)
display(set_heatmap(get_defense_stlats(improve_team_defense(real_pies, 0.20, shadows=True)), maxVal=1))

**20% Change**

,old_defense_stars,new_defense_stars,change_in_defense_stars
Tiana Takahashi,3.802671,4.604623,0.801952
Doc Anice,4.312819,5.049786,0.736966
Twofurious Puddles,3.610790,4.445614,0.834824
Case Lancaster,3.176257,4.212016,1.035759
Castillo Logan,2.996505,3.928175,0.931670
Haruta Byrd,1.671932,3.031379,1.359447
Yusef Puddles,3.712285,4.522409,0.810124
Mummy Cantburn,3.289650,4.105055,0.815405
Sheev Shriffle,0.873827,2.631009,1.757183
Doginic Burgertoes,1.538638,2.876826,1.338187


,Defense Rating,Omniscience,Tenaciousness,Watchfulness,Anticapitalism,Chasiness
Tiana Takahashi,0.920925,1.141378,0.628292,1.119010,1.221104,0.624413
Doc Anice,1.009957,1.029694,1.217277,1.154003,0.710123,0.857617
Twofurious Puddles,0.889123,1.047265,1.109861,0.460410,0.855028,0.580553
Case Lancaster,0.842403,1.167359,0.929116,0.461638,0.285625,1.160254
Castillo Logan,0.785635,0.949614,0.391755,0.917648,0.691621,1.019861
Haruta Byrd,0.606276,0.533077,0.633116,0.858868,0.210078,0.326473
Yusef Puddles,0.904482,0.716601,1.202048,0.515498,0.841809,1.138046
Mummy Cantburn,0.821011,0.791982,0.596224,0.716128,0.825545,1.055624
Sheev Shriffle,0.526202,0.338111,0.203537,0.458516,0.663167,1.130159
Doginic Burgertoes,0.575365,0.247259,1.064391,0.615999,0.318550,0.292532


In [21]:
display(Markdown("**40% Change**"))
infuse_defense_table, _, _ = improve_team_defense_table(real_pies, 0.40, shadows=True)
display(infuse_defense_table)
display(set_heatmap(get_defense_stlats(improve_team_defense(real_pies, 0.40, shadows=True)), maxVal=1))

**40% Change**

,old_defense_stars,new_defense_stars,change_in_defense_stars
Tiana Takahashi,3.802671,5.333581,1.530909
Doc Anice,4.312819,5.737533,1.424714
Twofurious Puddles,3.610790,5.189850,1.579060
Case Lancaster,3.176257,5.020516,1.844260
Castillo Logan,2.996505,4.720504,1.723999
Haruta Byrd,1.671932,3.908855,2.236923
Yusef Puddles,3.712285,5.255879,1.543594
Mummy Cantburn,3.289650,4.849029,1.559380
Sheev Shriffle,0.873827,3.582429,2.708603
Doginic Burgertoes,1.538638,3.813725,2.275087


,Defense Rating,Omniscience,Tenaciousness,Watchfulness,Anticapitalism,Chasiness
Tiana Takahashi,1.066716,1.341378,0.828292,1.319010,1.421104,0.824413
Doc Anice,1.147507,1.229694,1.417277,1.354003,0.910123,1.057617
Twofurious Puddles,1.037970,1.247265,1.309861,0.660410,1.055028,0.780553
Case Lancaster,1.004103,1.367359,1.129116,0.661638,0.485625,1.360254
Castillo Logan,0.944101,1.149614,0.591755,1.117648,0.891621,1.219861
Haruta Byrd,0.781771,0.733077,0.833116,1.058868,0.410078,0.526473
Yusef Puddles,1.051176,0.916601,1.402048,0.715498,1.041809,1.338046
Mummy Cantburn,0.969806,0.991982,0.796224,0.916128,1.025545,1.255624
Sheev Shriffle,0.716486,0.538111,0.403537,0.658516,0.863167,1.330159
Doginic Burgertoes,0.762745,0.447259,1.264391,0.815999,0.518550,0.492532


### Shadow Revoke
**Revoke a Player from your Team's Shadows.**

This would remove a player from our shadows by moving them to another team, without providing a replacement.

### Reform
**Re-roll an eligible Permanent Modification for a Player on your Team.**

Below is a list of all players with modifications that could be rerolled. It would be better to reroll modifications
with negative effects.

In [22]:
mod_players = [x for x in pies.lineup + pies.rotation if len(x.perm_attr) > 0]
mod_list = pandas.DataFrame()
for player in mod_players:
    data = pandas.Series({"mods": [x.title for x in player.perm_attr if x.id not in UNREFORMED_MODS]}, name=player.name)
    if len(data["mods"]) == 0:
        continue
    mod_list = mod_list.append(data)
mod_list

,mods
Beasley Day,[Minimalist]
Huber Frumple,[Seeker]
Ruslan Greatness,[Friend of Crows]
Jaxon Buckley,[Fire Eater]
Peanut Holloway,"[Honey Roasted, Negative]"
Henry Marshallow,[Maximalist]


### Magnify
**Magnify a Player on your Team. Runs batted in or allowed by this Player are multiplied.**

See below for amount of runs batted in/allowed for each player in the previous season. It is unknown what the mutliplier amount is.

In [23]:
table = get_batting_stats(pies.lineup, season=19)
table.filter(items=["runs_batted_in"])

,runs_batted_in
Eduardo Woodman,79.2
Beasley Day,60.0
Huber Frumple,71.6
Ruslan Greatness,74.7
Jaxon Buckley,81.0
Lang Richardson,43.1
Peanut Holloway,44.4
Usurper Violet,36.7


In [24]:
table = get_pitching_stats(pies.rotation, season=19)
table.filter(items=["runs_allowed"])

,runs_allowed
Henry Marshallow,48.1
Jacoby Podcast,74.0


___
## Blessings

Blessings are selected by raffle across all teams. Each blessing will be awarded to one random team, where teams with
more votes are more likely to be selected.

### TWO FREAKIN BATS
**Two random Players in your Team's Lineup with an available Item slot (empty or broken) will open Handcrafted Aluminum Bat Crates.**

[Two of them](https://www.youtube.com/watch?v=btHpHjabRcc). Aluminum bats increase Ground Friction while reducing
Musclitude and Buoyancy. See below for list of batters with an empty item slot.

In [25]:
free_items = [x for x in pies.lineup if (x.evolution + 1) - len([x for x in x.items if not x.is_broken]) > 0]
get_stars(free_items)

""


### Shadow Capes Supply Run
**All Players in your Team's Shadows with an available Item slot will open a Generic Cape Crate.**

Capes add Watchfulness, which is a defense stat. See below for list of shadow players with open item slots

In [26]:
free_items = [x for x in pies.shadows if (x.evolution + 1) - len([x for x in x.items if not x.is_broken]) > 0]
get_stars(free_items)

,Combined,Batting,Pitching,Baserunning,Defense
Adrian Melon,9.1,2.4,0.9,3.1,2.7
August Obrien,9.8,3.4,1.0,3.3,2.0
Mcfarland Vargas,8.7,1.4,1.5,3.8,2.1
Twofurious Puddles,11.5,2.6,3.5,1.8,3.6
Mummy Cantburn,9.0,1.4,2.3,2.0,3.3
Doginic Burgertoes,7.2,2.1,1.8,1.8,1.5
Nerd Pacheco,1.7,0.3,0.3,0.6,0.5
Yusef Puddles,11.6,2.4,2.5,2.9,3.7
Sheev Shriffle,8.4,3.4,1.9,2.2,0.9
Adeline Hammer,7.8,1.1,0.9,2.5,3.3


### Shots in the Dark
**Call the Negative Alternates (maintaining Star count) for 5 random Players in your Team's Shadows.**

A Negative Alternate is a player reroll (maintaining Combined Star value), while adding the Negative mod which makes
their eDensity Negative. See below for our current shadows, the target is random.

In [27]:
get_stars(pies.shadows)

,Combined,Batting,Pitching,Baserunning,Defense
Adrian Melon,9.1,2.4,0.9,3.1,2.7
August Obrien,9.8,3.4,1.0,3.3,2.0
Mcfarland Vargas,8.7,1.4,1.5,3.8,2.1
Twofurious Puddles,11.5,2.6,3.5,1.8,3.6
Mummy Cantburn,9.0,1.4,2.3,2.0,3.3
Doginic Burgertoes,7.2,2.1,1.8,1.8,1.5
Nerd Pacheco,1.7,0.3,0.3,0.6,0.5
Yusef Puddles,11.6,2.4,2.5,2.9,3.7
Sheev Shriffle,8.4,3.4,1.9,2.2,0.9
Adeline Hammer,7.8,1.1,0.9,2.5,3.3


### Couldn't Hurt
**Call the Negative Alternates (maintaining Star count) for the 2 worst Players on your Team's Active Roster.**

A Negative Alternate is a player reroll (maintaining Combined Star value), while adding the Negative mod which makes
their eDensity Negative. See below for the 2 worst players on the team.

In [28]:
worst_pies = sort_overall(pies, num=2)
get_stars(worst_pies)

,Combined,Batting,Pitching,Baserunning,Defense
Usurper Violet,11.7,2.9,3.2,1.7,3.9
Lang Richardson,11.2,3.2,0.7,3.6,3.7


### Green Light
**One Player on all Teams in your Division will play 50% better in Polarity Plus Weather and 50% worse in Polarity Minus Weather.**

~Random~. Playing worse in Polarity Minus is good if it hits a batter, because you dont want to score when it would result in unruns.

### Sponge Blob
**Give all Players with Ego on your Team's Roster the Sponge modification.**

See below for list of players with any level of Ego. The effect of the Sponge modification is unknown.

In [29]:
egod = [x for x in pies.lineup + pies.rotation + pies.shadows if any(mod in ("EGO1", "EGO2", "EGO3", "EGO4") for mod in x._perm_attr_ids)]
get_stars(egod)

,Combined,Batting,Pitching,Baserunning,Defense
Jaxon Buckley,17.0,6.6,3.7,3.5,3.3


### Heavy-Handed & Light-Handed
**Your Team becomes Heavy-Handed, making all Items held by Players on the Team positively eDense.**

**Your Team becomes Light-Handed, making all Items held by Players on the Team negatively eDense.**

Items are currently not contributing to eDensity for either players or teams, so the exact effect of these is unknown.

### Strange Attractor
**Recruit a Random Player in the League with the Attractor Modification to your Team.**

See below for list of all active players with Attractor. This will not remove an existing player from our team.

Notes:
* This lists the False Stars, they will play using their player attributes instead.

In [30]:
attractors = []
for team in all_teams:
    attractors.extend([x for x in team.lineup + team.rotation if "ATTRACTOR" in x._perm_attr_ids])
display(get_stars(attractors, include_team=True))

,,Combined,Batting,Pitching,Baserunning,Defense
,Team,,,,,
Liquid Friend,Jazz Hands,21.5,6.8,4.2,5.2,5.3
Alyssa Harrell,Firefighters,22.9,5.8,6.7,4.4,6.0
Chorby Short,Flowers,25.3,5.3,7.6,6.9,5.5
Forrest Best,Steaks,29.1,5.9,8.9,7.0,7.2
Tiana Wheeler,Magic,12.6,4.3,1.5,1.8,4.9


### Fringe Benefit
**Recruit a Random Player with the Perk Modification to your Team.**

See below for list of all active players with Perk. This will not remove an existing player from our team. Note that
Perk adds to eDensity.

In [31]:
attractors = []
for team in all_teams:
    attractors.extend([x for x in team.lineup + team.rotation  if "PERK" in x._perm_attr_ids])
display(get_stars(attractors, include_team=True))

,,Combined,Batting,Pitching,Baserunning,Defense
,Team,,,,,
Baby Doyle,Jazz Hands,17.0,5.5,2.9,3.7,4.8
Glabe Moon,Flowers,16.5,3.6,2.0,4.9,6.0
Parker Meng,Worms,12.9,1.9,4.7,2.7,3.6
Sandie Turner,Millennials,11.3,3.4,3.1,3.1,1.7
Holden Stanton,Steaks,11.0,3.3,0.7,3.0,4.0
Knight Triumphant,Steaks,20.8,6.2,5.0,4.6,5.1
Commissioner Vapor,Moist Talkers,15.8,5.3,3.1,4.4,3.0
Beans McBlase,Moist Talkers,17.0,4.4,4.1,2.8,5.7
Cornelius Games,Shoe Thieves,10.1,1.4,4.0,1.3,3.4


### Pitch Runner
**Move the Best Baserunner in your Team's Rotation to a random spot in your Team's Lineup.**

This would move the following player to our lineup.

In [32]:
temp_pies = pies.rotation.copy()
temp_pies.sort(key=lambda x: x.get_baserunning_rating(), reverse=True)
display(get_stars(temp_pies[0]))
set_heatmap(get_batting_stlats([temp_pies[0]] + pies.lineup), maxVal=1.0)

,Combined,Batting,Pitching,Baserunning,Defense
Henry Marshallow,14.8,3.6,3.8,2.7,4.7


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Henry Marshallow,0.713297,0.399719,1.150909,0.596577,1.045685,0.429725,0.877213,0.001000,1.095975
Eduardo Woodman,1.126442,1.345315,1.152985,0.970888,0.876160,0.283687,0.749895,0.001000,1.025183
Beasley Day,0.904594,1.045160,0.904890,0.612953,0.707377,0.211371,0.733784,0.001000,1.009991
Huber Frumple,0.887107,0.534025,1.345741,0.959816,1.242462,0.188405,0.708998,0.005052,1.138288
Ruslan Greatness,0.951096,0.622589,1.145689,1.569998,1.470457,0.001000,1.311486,0.001000,0.683552
Jaxon Buckley,1.318562,1.567382,1.042101,1.746709,1.930045,0.001000,1.986793,0.001000,1.393504
Lang Richardson,0.632443,0.812722,0.400621,0.622843,0.941821,0.205246,0.498400,0.001000,0.551015
Peanut Holloway,0.848814,0.769919,0.943909,0.997017,0.943055,0.006915,0.197960,0.773163,0.974957
Usurper Violet,0.579715,0.894648,0.293503,1.022726,0.869795,0.569699,0.283561,0.100000,0.487176


### Slowpoke
**Move the Worst Baserunner in your Team's Lineup to a random spot in your Team's Rotation.**

This would move the following player to our rotation.

In [33]:
temp_pies = pies.lineup.copy()
temp_pies.sort(key=lambda x: x.get_baserunning_rating())
display(get_stars(temp_pies[0]))
set_heatmap(get_pitching_stlats([temp_pies[0]] + pies.rotation), maxVal=1.0)

,Combined,Batting,Pitching,Baserunning,Defense
Usurper Violet,11.7,2.9,3.2,1.7,3.9


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Usurper Violet,0.635537,0.916861,0.580602,0.980022,0.172289,0.582188,0.847305
Henry Marshallow,0.768310,0.491270,1.171077,1.428524,0.822605,0.808351,0.876106
Jacoby Podcast,0.954583,0.824371,1.141648,0.868777,1.157278,1.154442,0.513636
Lucy Tokkan,0.741748,0.901966,0.995562,0.263758,0.973020,0.180980,0.321805


### Stuck in the Freezer
**A Random Player in your Team's Rotation will have their Coldness Boosted by 40%.**

Coldness is currently unknown, though potentially prevents stolen bases. See below for change in player stars and stlats.

In [34]:
new_team = []
for player in real_pies.rotation:
    new_team.append(player.simulated_copy(buffs={"coldness": 0.4}))
display(get_stars(new_team))
display(set_heatmap(get_pitching_stlats(new_team), maxVal=1))

,Combined,Batting,Pitching,Baserunning,Defense
Henry Marshallow,14.9,3.6,3.9,2.7,4.7
Jacoby Podcast,12.8,2.7,4.8,1.8,3.5
Lucy Tokkan,10.0,2.1,3.8,1.1,2.9


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.776071,0.491270,1.171077,1.428524,0.822605,1.208351,0.876106
Jacoby Podcast,0.961709,0.824371,1.141648,0.868777,1.157278,1.554442,0.513636
Lucy Tokkan,0.763694,0.901966,0.995562,0.263758,0.973020,0.580980,0.321805


### Shot Caller
**A Random Player in your Team's Lineup will be 40% better at landing Home Runs in a Big Bucket.**

See below for list of teams that currently have Big Buckets in their Stadium.

In [35]:
stadiums = Stadium.load_all()
table = pandas.DataFrame()
for s in stadiums.values():
    if "BIG_BUCKET" in s._mods_ids:
        table = table.append(pandas.Series({"Stadium Name": s.name}, name=s.team_id.nickname))
table

,Stadium Name
Firefighters,The Fire Safety Awareness Center
Breath Mints,Kansas City FreshDome
Wild Wings,Basin of Mexico Park
Magic,Yellowstone National (Ball)Park
Tacos,Al Pastor Memorial Park
Spies,An Undisclosed Location
Steaks,George Fourman Stadium
Tigers,Sixth Circle Stadium
Dale,Miami International Arena
Pies,Tastycake Stadium


### Cool Off
**All Players in your Team's Rotation will gain 20% Coldness but lose 5% Ruthlessness.**

Coldness is unknown, and Ruthlessness is more correlated to increased strikes and fewer balls thrown. See below for
change in team stats.

In [36]:
new_team = []
for player in real_pies.rotation:
    new_team.append(player.simulated_copy(buffs={"coldness": 0.2, "ruthlessness": -0.05}))
display(set_heatmap(get_pitching_stlats(new_team), maxVal=1))

,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.759201,0.491270,1.121077,1.428524,0.822605,1.008351,0.876106
Jacoby Podcast,0.941388,0.824371,1.091648,0.868777,1.157278,1.354442,0.513636
Lucy Tokkan,0.740264,0.901966,0.945562,0.263758,0.973020,0.380980,0.321805


### Global Guts
**All Players in your Team's Lineup will gain 20% Moxie but lose 5% Thwackability.**

Moxie correlates to better plate discipline (more walks), and thwackability correlates to better contact. See below for
change in team stats.

In [37]:
new_team = []
for player in real_pies.lineup:
    new_team.append(player.simulated_copy(buffs={"moxie": 0.2, "thwackability": -0.05}))
display(set_heatmap(get_batting_stlats(new_team), maxVal=1))

,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,1.128883,1.295315,1.152985,0.970888,1.076160,0.283687,0.749895,0.001000,1.025183
Beasley Day,0.905968,0.995160,0.904890,0.612953,0.907377,0.211371,0.733784,0.001000,1.009991
Huber Frumple,0.866752,0.484025,1.345741,0.959816,1.442462,0.188405,0.708998,0.005052,1.138288
Ruslan Greatness,0.932509,0.572589,1.145689,1.569998,1.670457,0.001000,1.311486,0.001000,0.683552
Jaxon Buckley,1.313362,1.517382,1.042101,1.746709,2.130045,0.001000,1.986793,0.001000,1.393504
Lang Richardson,0.627541,0.762722,0.400621,0.622843,1.141821,0.205246,0.498400,0.001000,0.551015
Peanut Holloway,0.841145,0.719919,0.943909,0.997017,1.143055,0.006915,0.197960,0.773163,0.974957
Usurper Violet,0.577051,0.844648,0.293503,1.022726,1.069795,0.569699,0.283561,0.100000,0.487176


### Backup or Down
**Flip the eDensity of the most eDense Player in your Team's Shadows.**

This would make the most dense shadows player have the Negative modification, making their eDensity negative.

In [38]:
temp_pies = pies.shadows.copy()
temp_pies.sort(key=lambda x: x.e_density, reverse=True)
display(f"{temp_pies[0].name} eDensity change: {temp_pies[0].e_density} to {-temp_pies[0].e_density}")
display(f"Total Team eDensity change: {pies.e_density} to {pies.e_density - 2*temp_pies[0].e_density}")

'Doc Anice eDensity change: 39.81715027609057 to -39.81715027609057'

'Total Team eDensity change: 1350.7216513024707 to 1271.0873507502895'

### Complete 180

**Flip the eDensity of the most eDense Player in your Team's Rotation.**

This would make the most dense pitcher have the Negative modification, making their eDensity negative.

In [39]:
temp_pies = pies.rotation.copy()
temp_pies.sort(key=lambda x: x.e_density, reverse=True)
display(f"{temp_pies[0].name} eDensity change: {temp_pies[0].e_density} to {-temp_pies[0].e_density}")
display(f"Total Team eDensity change: {pies.e_density} to {pies.e_density - 2*temp_pies[0].e_density}")

'Henry Marshallow eDensity change: 30.738460279675117 to -30.738460279675117'

'Total Team eDensity change: 1350.7216513024707 to 1289.2447307431205'

### Head over Tails

**Flip the eDensity of the most eDense Player in your Team's Lineup.**

This would make the most dense batter have the Negative modification, making their eDensity negative.

In [40]:
temp_pies = pies.lineup.copy()
temp_pies.sort(key=lambda x: x.e_density, reverse=True)
display(f"{temp_pies[0].name} eDensity change: {temp_pies[0].e_density} to {-temp_pies[0].e_density}")
display(f"Total Team eDensity change: {pies.e_density} to {pies.e_density - 2*temp_pies[0].e_density}")

'Jaxon Buckley eDensity change: 44.35894171251668 to -44.35894171251668'

'Total Team eDensity change: 1350.7216513024707 to 1262.0037678774374'

### Low Five

**Flip the eDensity of the five least eDense Players on your Team's Roster.**

See below for list of players and their eDensity impact.

In [41]:
temp_pies = pies.lineup.copy() + pies.rotation.copy() + pies.shadows.copy()
temp_pies.sort(key=lambda x: x.e_density)
table = pandas.DataFrame()
for p in temp_pies[0:5]:
    table = table.append(pandas.Series({"eDensity": p.e_density}, name=p.name))
display(table)
density_sum = sum([x.e_density for x in temp_pies[0:5]])
display(f"Total Team eDensity change: {pies.e_density} to {pies.e_density - 2*density_sum}")

,eDensity
Peanut Holloway,-21.180626
Nerd Pacheco,10.323060
Bambi Perez,14.032362
Mummy Cantburn,16.639602
Jesús Rodriguez,19.310689


'Total Team eDensity change: 1350.7216513024707 to 1272.4714791949295'

### Slow Clap

**The first Player in your Team's Lineup will gain the Slow Build mod.**

Slow Build makes players bat 1% better for each At Bat in a game, resetting at the beginning of each game.

### States of Play

**Give all Elsewhere or Scattered Players on your Team's Roster the Undefined mod.**

Undefined makes a player play 50-100% better while Scattered. See below for list of Elsewhere of Scattered players.

In [42]:
scattered = [x for x in pies.lineup + pies.rotation + pies.shadows if any(mod in ("SCATTERED", "ELSEWHERE") for mod in x._perm_attr_ids)]
get_stars(scattered)

""


### Solo Stroll

**The worst Batter in your Team's Lineup will draw a walk on 3 balls instead of 4.**

This would add permanent Walk in the Park to our worst batter, listed below.

In [43]:
get_stars(worst_batter)

,Combined,Batting,Pitching,Baserunning,Defense
Usurper Violet,11.7,2.9,3.2,1.7,3.9


### Targeted Evolution

**A random pitcher and a random hitter on your Team will Advance.**

An Advanced player will gain one evolution level, increasing their minimum star level by one for all stats and raising
their stars to the minimum if they are lower than that. It also allows players to hold an additional item.

## Gifts

Gifts are awarded to teams based on contributions from other teams. Each team creates a Wishlist of items that they
would like, and the top items on the wishlist are awarded during the Latesiesta.

### eDense Infusion

**Ratings Boost! Add the eDense Element to 20% of the non-eDense Items held by Players on your Team.**

It is unknown what item eDensity is or how it effects the sim.

### Collector's Editions

Collectors Editions move a Legendary player to the team. Replicas act as copies of that player and will turn to Dust
after elections.

Special Notes:
* Nagomi has fake Attractor stars, this analysis uses the player attributes.
* Chorby is still unstable, and is likely to get incinerated if possible.
* Chorby replicas have 0 soul, so are safe from consumers

In [44]:
york = Player.find_by_name("York Silk")
nagomi = Player.find_by_name("Nagomi Mcdaniel")
nagomi.batting_rating = None
nagomi.baserunning_rating = None
nagomi.defense_rating = None
aldon = Player.find_by_name("Aldon Cashmoney")
goodwin = Player.find_by_name("Goodwin Morin")
val = Player.find_by_name("Valentine Games")
pm = Player.find_by_name("Pitching Machine")
chorby = Player.find_by_name("Chorby Soul")

display(Markdown("#### Batting"))
display(set_heatmap(get_batting_stlats([chorby, val, aldon, goodwin, pm] + pies.lineup), maxVal=1.5))

display(Markdown("Adding Chorby Soul"))
display(pandas.DataFrame(move_player(pies, chorby, position='lineup')))

display(Markdown("Adding Valentine Games"))
display(pandas.DataFrame(move_player(pies, val, position='lineup')))

display(Markdown("Adding Aldon Cashmoney"))
display(pandas.DataFrame(move_player(pies, aldon, position='lineup')))

display(Markdown("Adding Goodwin Morin"))
display(pandas.DataFrame(move_player(pies, goodwin, position='lineup')))

display(Markdown("Adding Pitching Machine"))
display(pandas.DataFrame(move_player(pies, pm, position='lineup')))

display(Markdown("#### Pitching"))
display(set_heatmap(get_pitching_stlats([nagomi, chorby] + pies.rotation), maxVal=1.5))

display(Markdown("Adding Nagomi McDaniel"))
display(pandas.DataFrame(move_player(pies, nagomi, position='rotation')))

display(Markdown("Adding Chorby Soul"))
display(pandas.DataFrame(move_player(pies, chorby, position='rotation')))

#### Batting

,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Chorby Soul,0.001820,0.001000,0.001000,0.001000,0.001000,0.990000,0.001000,0.999000,0.001000
Valentine Games,1.322240,1.237714,1.407875,1.829484,1.477367,0.001000,1.685680,0.001000,1.274472
Aldon Cashmoney,0.998344,1.058616,1.070545,0.723468,1.110157,0.342420,0.728757,0.158430,0.415462
Goodwin Morin,1.116151,1.011205,1.054999,1.804185,1.724130,0.001000,1.146978,0.054253,1.040647
Pitching Machine,0.968126,0.637427,1.256693,1.400732,1.401861,0.154714,1.164864,0.001000,1.399386
Eduardo Woodman,1.126442,1.345315,1.152985,0.970888,0.876160,0.283687,0.749895,0.001000,1.025183
Beasley Day,0.904594,1.045160,0.904890,0.612953,0.707377,0.211371,0.733784,0.001000,1.009991
Huber Frumple,0.887107,0.534025,1.345741,0.959816,1.242462,0.188405,0.708998,0.005052,1.138288
Ruslan Greatness,0.951096,0.622589,1.145689,1.569998,1.470457,0.001000,1.311486,0.001000,0.683552
Jaxon Buckley,1.318562,1.567382,1.042101,1.746709,1.930045,0.001000,1.986793,0.001000,1.393504


Adding Chorby Soul

,Pies
batting_change,-10.0475%
pitching_change,+0.0000%
baserunning_change,-7.6039%
defense_change,-7.8861%


Adding Valentine Games

...

,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Nagomi Mcdaniel,0.862040,0.917836,0.608354,0.703359,1.145577,0.994793,0.560973
Chorby Soul,0.000299,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000
Henry Marshallow,0.768310,0.491270,1.171077,1.428524,0.822605,0.808351,0.876106
Jacoby Podcast,0.954583,0.824371,1.141648,0.868777,1.157278,1.154442,0.513636
Lucy Tokkan,0.741748,0.901966,0.995562,0.263758,0.973020,0.180980,0.321805


Adding Nagomi McDaniel

,Pies
batting_change,+0.0000%
pitching_change,+1.0123%
baserunning_change,+0.0000%
defense_change,+0.0000%


Adding Chorby Soul

,Pies
batting_change,+0.0000%
pitching_change,-20.5312%
baserunning_change,+0.0000%
defense_change,+0.0000%


### Solo Editions

These gifts provide a random player with a benefit for the lateseason and postseason.

* Ambitious: Player will overperform in the postseason.
* Unambitious: Unknown, presumably the opposite of Ambitious.

### Team Editions

These gifts provide a team-wide benefit for the lateseason and postseason.

* Late to the Party: Team will overperform in the Lateseason.
* Early to the Party: Unknown, presumably the opposite of Late to the Party.
* Fireproof: Team will be safe from incinerations. Pies only have 1 Solar Eclipse game in the lateseason.
* Soundproof: Team will be safe from Feedback. Pies have 2 Feedback weather games in the lateseason.
* Gravity: Team will be safe from Reverb. Pies only have 1 Reverb weather game in the lateseason.

### Handcrafted Drops

These will provide Items which have additional stat increases and random special effects.

|Type | Attribute | Stat |
| --- | --- | --- |
| Bat | +Thwackability | Batting |
| Cap | +Unthwackability | Pitching |
| Phone | ??? | ??? |
| Quill | ??? | ??? |

See below for a list of players that can receive them.

The Bats will have Subtractor, converting Runs to UnRuns and vice-versa. The Caps will have Underhanded which will
convert earned Home Runs to UnRuns.

**Bat**

In [45]:
free_items_lineup = [x for x in pies.lineup if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items_lineup)

""


**Cap, Quill**

In [46]:
free_items_rotation= [x for x in pies.rotation if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items_rotation)

,Combined,Batting,Pitching,Baserunning,Defense
Lucy Tokkan,9.9,2.1,3.7,1.1,2.9


**Phone**

In [47]:
free_items = [x for x in pies.lineup + pies.rotation if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items)

,Combined,Batting,Pitching,Baserunning,Defense
Lucy Tokkan,9.9,2.1,3.7,1.1,2.9


### Soul Patches

**Back by Popular Demand! Give 1 Soul to your least Soul Full player. Repeat 10 times.**

See below for final Soul totals. Low soul is dangerous, as Players will become Redacted if their soul hits 0 and
Consumers will drain 1 soul per chomp.

In [48]:
players = deepcopy(pies.rotation) + deepcopy(pies.lineup)
for i in range(0, 10):
    players.sort(key=lambda x: x.soul)
    worst = players[0]
    worst.soul += 1
    players = [x for x in players if x.id != worst.id] + [worst]
players.sort(key=lambda x: x.soul)
x = pandas.DataFrame()
for p in players:
    old = [x for x in pies.rotation + pies.lineup if x.id == p.id][0]
    x = x.append(pandas.Series({"Soul Added": p.soul - old.soul,"Resulting Soul": p.soul}, name=p.name))
x

,Resulting Soul,Soul Added
Usurper Violet,5.0,1.0
Peanut Holloway,5.0,2.0
Lang Richardson,6.0,0.0
Henry Marshallow,6.0,1.0
Jacoby Podcast,6.0,2.0
Eduardo Woodman,6.0,2.0
Ruslan Greatness,6.0,2.0
Beasley Day,7.0,0.0
Huber Frumple,7.0,0.0
Jaxon Buckley,7.0,0.0


### Bargain Bin

**Give a random Player on your Team the Best Item from the Bargain Bin!**

The Bargain Bin will retrieve one random previously dropped item. Below is a list of all non-base items not currently
held by players.

In [49]:
all_items = Item.load_all()
rare_items = [x for x in all_items.values() if len(x.adjustments) > 1]
players = Player.load_all()
carried_items = []
for p in players.values():
    carried_items.extend(p.items)
dropped_items = [x for x in rare_items if x not in carried_items]
dropped_items.sort(key=lambda x: len(x.adjustments), reverse=True)

table = []
for i in dropped_items:
    mods = [Modification.load_one(a["mod"]).title for a in i.adjustments if a["type"] == 0]
    table.append(pandas.Series({"Durability": i.durability, "Modifications": mods}, name=i.name))
pandas.DataFrame(table)

,Durability,Modifications
Weird Plant-Based Bat,3,[]
Hearty Rubber Cap,3,[]
Cool Holey Shoes,3,[]
Sharp Jersey of Charisma,3,[]
Sharp Glove of Wisdom,3,[]
Rubber Skate Bat of Strength,4,[Pro Skater]
Aluminum Blood Sunglasses,3,[]
Sharp Leg Bat,3,[]
Holey Sunglasses of Strength,3,[]
Weird Shoes,2,[]


---
## Appendix
* [Description of Attributes](https://www.blaseball.wiki/w/Player_Attributes)
* [Stlat Viewer](https://slavfox.space/abslve/?foreboding-kaleidoscope#PHIL)
* [Historical Player Graphs](http://yoori.space/hloroscopes/)
